# Module biogeme.optimization 

## Examples of use of each function

This webpage is for programmers who need examples of use of the functions of the class. The examples are designed to illustrate the syntax. They do not correspond to any meaningful model. For examples of models, visit  [biogeme.epfl.ch](http://biogeme.epfl.ch).

In [1]:
import datetime
print(datetime.datetime.now())

2023-04-14 12:01:01.097739


In [2]:
import biogeme.version as ver
print(ver.getText())

biogeme 3.2.11 [2023-04-14]
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import numpy as np
import pandas as pd

In [4]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [5]:
import biogeme.optimization as opt
import biogeme.biogeme as bio
import biogeme.database as db
import biogeme.models as models
from biogeme.expressions import Beta, Variable
import biogeme.logging as blog


In [6]:
logger = blog.get_screen_logger(blog.INFO)

# Biogeme example

In [7]:
df = pd.DataFrame({'Person': [1, 1, 1, 2, 2],
                   'Exclude': [0, 0, 1, 0, 1],
                   'Variable1': [1, 2, 3, 4, 5],
                   'Variable2': [10, 20, 30,40, 50],
                   'Choice': [1, 2, 3, 1, 2],
                   'Av1': [0, 1, 1, 1, 1],
                   'Av2': [1, 1, 1, 1, 1],
                   'Av3': [0, 1, 1, 1, 1]})
myData = db.Database('test', df)

Choice = Variable('Choice')
Variable1 = Variable('Variable1')
Variable2 = Variable('Variable2')
beta1 = Beta('beta1', 0, None, None, 0)
beta2 = Beta('beta2', 0, None, None, 0)
V1 = beta1 * Variable1
V2 = beta2 * Variable2
V3 = 0
V ={1: V1,2: V2,3: V3}

likelihood = models.loglogit(V, av=None, i=Choice)
myBiogeme = bio.BIOGEME(myData, likelihood)
myBiogeme.modelName = 'simpleExample'
myBiogeme.saveIterations = False
myBiogeme.generate_html = False
myBiogeme.generate_pickle = False
print(myBiogeme)

Parameters read from biogeme.toml 


simpleExample: database [test]{'loglike': _bioLogLogitFullChoiceSet[choice=Choice]U=(1:(beta1(init=0) * Variable1), 2:(beta2(init=0) * Variable2), 3:`0.0`)av=(1:`1.0`, 2:`1.0`, 3:`1.0`)}


In [8]:
f, g, h, gdiff, hdiff = myBiogeme.checkDerivatives(beta=myBiogeme.beta_values_dict_to_list(), verbose=True)

In [9]:
gdiff

array([6.03961326e-07, 6.11038797e-05])

In [10]:
hdiff

array([[ 8.31415072e-07, -4.16670705e-05],
       [-4.27475899e-06,  8.33270355e-04]])

## scipy

This is the optimization algorithm from scipy. 

In [11]:
myBiogeme.algorithm_name = 'scipy'
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: scipy 
Minimize with tol 1e-07 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144546,0.366198,0.394720,0.693049
beta2,0.023502,0.034280,0.685574,0.492982


In [12]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	scipy.optimize
Cause of termination:	CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Number of iterations:	13
Number of function evaluations:	16
Optimization time:	0:00:00.003088


## Newton with linesearch

In [13]:
myBiogeme.algorithm_name = 'LS-newton'
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: Newton with line search [LS-newton] 
** Optimization: Newton with linesearch 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144545,0.366198,0.394720,0.693050
beta2,0.023502,0.034280,0.685573,0.492982


In [14]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with line search
Relative gradient:	3.43067908440321e-07
Number of iterations:	3
Number of function evaluations:	10
Number of gradient evaluations:	10
Number of hessian evaluations:	4
Cause of termination:	Relative gradient = 3.4e-07 <= 6.1e-06
Optimization time:	0:00:00.001855


Changing the requested precision


In [15]:
myBiogeme.tolerance = 0.1
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: Newton with line search [LS-newton] 
** Optimization: Newton with linesearch 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144317,0.365691,0.394641,0.693108
beta2,0.023428,0.034256,0.683887,0.494047


In [16]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with line search
Relative gradient:	0.015380029144206175
Number of iterations:	2
Number of function evaluations:	7
Number of gradient evaluations:	7
Number of hessian evaluations:	3
Cause of termination:	Relative gradient = 0.015 <= 0.1
Optimization time:	0:00:00.001757


## Newton with trust region

In [17]:
myBiogeme.algorithm_name = 'TR-newton'
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: Newton with trust region [TR-newton] 
** Optimization: Newton with trust region 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144317,0.365691,0.394641,0.693108
beta2,0.023428,0.034256,0.683887,0.494047


In [18]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with trust region
Relative gradient:	0.014746524905601738
Cause of termination:	Relative gradient = 0.015 <= 0.1
Number of iterations:	2
Number of function evaluations:	4
Number of gradient evaluations:	3
Number of hessian evaluations:	3
Optimization time:	0:00:00.001527


We illustrate the parameters. We use the truncated conjugate gradient instead of dogleg for the trust region subproblem, starting with a small trust region of radius 0.001, and a maximum of 3 iterations.

In [19]:
myBiogeme.dogleg = False
myBiogeme.initial_radius=0.001
myBiogeme.maxiter = 3
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: Newton with trust region [TR-newton] 
** Optimization: Newton with trust region 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.000053,0.308910,0.000170,0.999864
beta2,0.007000,0.030019,0.233173,0.815627


In [20]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with trust region
Relative gradient:	2.0182963630997235
Cause of termination:	Maximum number of iterations reached: 3
Number of iterations:	3
Number of function evaluations:	6
Number of gradient evaluations:	4
Number of hessian evaluations:	4
Optimization time:	0:00:00.001437


Changing the requested precision


In [21]:
myBiogeme.tolerance=0.1
myBiogeme.maxiter = 1000
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: Newton with trust region [TR-newton] 
** Optimization: Newton with trust region 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.111901,0.357903,0.312658,0.754541
beta2,0.021615,0.032531,0.664435,0.506412


In [22]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with trust region
Relative gradient:	0.04203221663984582
Cause of termination:	Relative gradient = 0.042 <= 0.1
Number of iterations:	7
Number of function evaluations:	14
Number of gradient evaluations:	8
Number of hessian evaluations:	8
Optimization time:	0:00:00.002670


## BFGS with line search

In [23]:
myBiogeme.algorithm_name = 'LS-BFGS'
myBiogeme.tolerance=1.0e-6
myBiogeme.maxiter = 1000
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: BFGS with line search [LS-BFGS] 
** Optimization: BFGS with line search 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144545,0.366198,0.394720,0.693050
beta2,0.023502,0.034280,0.685573,0.492982


In [24]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Inverse BFGS with line search
Relative gradient:	5.933646980468922e-07
Cause of termination:	Relative gradient = 5.9e-07 <= 1e-06
Number of iterations:	5
Number of function evaluations:	28
Number of gradient evaluations:	6
Optimization time:	0:00:00.002667


## BFGS with trust region

In [25]:
myBiogeme.algorithm_name = 'TR-BFGS'
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: BFGS with trust region [TR-BFGS] 
** Optimization: BFGS with trust region 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144546,0.366198,0.394720,0.693049
beta2,0.023502,0.034280,0.685574,0.492982


In [26]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	BFGS with trust region
Relative gradient:	2.1726273024450062e-09
Cause of termination:	Relative gradient = 2.2e-09 <= 1e-06
Number of iterations:	11
Number of function evaluations:	23
Number of gradient evaluations:	12
Optimization time:	0:00:00.003441


## Newton/BFGS with trust region for simple bounds

This is the default algorithm used by Biogeme. It is the implementation of the algorithm proposed by [Conn et al. (1988)](https://www.ams.org/journals/mcom/1988-50-182/S0025-5718-1988-0929544-3/S0025-5718-1988-0929544-3.pdf).

In [27]:
myBiogeme.algorithm_name = 'simple_bounds'
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: Newton with trust region for simple bounds 
Proportion of Hessian calculation: 100.0% 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144544,0.366197,0.394715,0.693053
beta2,0.023501,0.034280,0.685572,0.492983


In [28]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Newton with trust region for simple bound constraints
Proportion analytical hessian:	100.0%
Relative projected gradient:	2.6098345611870616e-06
Relative change:	7.891033303175643e-08
Number of iterations:	6
Number of function evaluations:	19
Number of gradient evaluations:	7
Number of hessian evaluations:	7
Cause of termination:	Relative change = 7.89e-08 <= 1e-05
Optimization time:	0:00:00.003293


When the second derivatives are too computationally expensive to calculate, it is possible to avoid calculating them at each successful iteration. The parameter `second_derivatives` allows to control that.

In [29]:
myBiogeme.second_derivatives = 0.5
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: Hybrid Newton 50.0%/BFGS with trust region for simple bounds 
Proportion of Hessian calculation: 50.0% 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144546,0.366198,0.394720,0.693049
beta2,0.023502,0.034280,0.685574,0.492982


In [30]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Hybrid Newton [50.0%] with trust region for simple bound constraints
Proportion analytical hessian:	50.0%
Relative projected gradient:	1.7714802103915912e-07
Relative change:	5.4841813972028586e-09
Number of iterations:	7
Number of function evaluations:	22
Number of gradient evaluations:	8
Number of hessian evaluations:	4
Cause of termination:	Relative change = 5.48e-09 <= 1e-05
Optimization time:	0:00:00.003406


If the parameter is set to zero, the second derivatives are not used at all, and the algorithm relies only on the BFGS update.

In [31]:
myBiogeme.second_derivatives = 0.0
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: BFGS with trust region for simple bounds 
Proportion of Hessian calculation: 0.0% 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144546,0.366198,0.394720,0.693049
beta2,0.023502,0.034280,0.685573,0.492982


In [32]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	BFGS with trust region for simple bound constraints
Proportion analytical hessian:	0.0%
Relative projected gradient:	1.6091903729961065e-06
Relative change:	5.793383296454069e-06
Number of iterations:	7
Number of function evaluations:	22
Number of gradient evaluations:	8
Number of hessian evaluations:	0
Cause of termination:	Relative change = 5.79e-06 <= 1e-05
Optimization time:	0:00:00.003161


There are shortcuts to call the BFGS and the Newton versions

In [33]:
myBiogeme.algorithm_name = 'simple_bounds_newton'
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: Newton with simple bounds [simple_bounds_newton]. 
Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: Newton with trust region for simple bounds 
Proportion of Hessian calculation: 100.0% 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144544,0.366197,0.394715,0.693053
beta2,0.023501,0.034280,0.685572,0.492983


In [34]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Newton with trust region for simple bound constraints
Proportion analytical hessian:	100.0%
Relative projected gradient:	2.6098345611870616e-06
Relative change:	7.891033303175643e-08
Number of iterations:	6
Number of function evaluations:	19
Number of gradient evaluations:	7
Number of hessian evaluations:	7
Cause of termination:	Relative change = 7.89e-08 <= 1e-05
Optimization time:	0:00:00.003111


In [35]:
myBiogeme.algorithm_name = 'simple_bounds_BFGS'
results = myBiogeme.estimate()
results.getEstimatedParameters()

Optimization algorithm: BFGS with simple bounds [simple_bounds_BFGS]. 
Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: BFGS with trust region for simple bounds 
Proportion of Hessian calculation: 0.0% 


,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144546,0.366198,0.394720,0.693049
beta2,0.023502,0.034280,0.685573,0.492982


In [36]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	BFGS with trust region for simple bound constraints
Proportion analytical hessian:	0.0%
Relative projected gradient:	1.6091903729961065e-06
Relative change:	5.793383296454069e-06
Number of iterations:	7
Number of function evaluations:	22
Number of gradient evaluations:	8
Number of hessian evaluations:	0
Cause of termination:	Relative change = 5.79e-06 <= 1e-05
Optimization time:	0:00:00.003046
